In [19]:
import pandas as pd
import numpy as np
import folium
import json
import ast


df = pd.read_csv("data/usa-2016-presidential-election-by-county.csv", sep=';')




In [27]:
ast.literal_eval(df['Geo Shape'].iloc[50])

{'type': 'Polygon',
 'coordinates': [[[-90.6777529996, 45.7244609999],
   [-90.6778100004, 45.8448300001],
   [-90.6786610001, 45.8684569997],
   [-90.677411, 45.874957],
   [-90.6771620002, 45.8853570002],
   [-90.6788649996, 45.9389549997],
   [-90.6769680003, 45.981554],
   [-90.4683000002, 45.9811799999],
   [-90.043958, 45.9819480002],
   [-90.0428059999, 45.8749519997],
   [-90.0451979996, 45.8176520004],
   [-90.0452100001, 45.7747569999],
   [-90.0430539996, 45.7160430002],
   [-90.0436639999, 45.5863410003],
   [-90.0422729996, 45.3818660003],
   [-90.1127280003, 45.38264],
   [-90.1592629996, 45.3814839998],
   [-90.2903139996, 45.380903],
   [-90.4581800003, 45.3769039999],
   [-90.6787390001, 45.3777700003],
   [-90.6779800002, 45.4936300003],
   [-90.6788600003, 45.5010499999],
   [-90.67981, 45.53955],
   [-90.6777529996, 45.7244609999]]]}

In [32]:
#Store geodata and add County ids

with open('counties.json', 'w') as geoData:
    geoDict = {
        "type":"FeatureCollection",
        "features":[]
    }

    for index, row in df.iterrows():
        if row['Geo Shape'] is not np.nan:
            geoDict['features'].append(
                {
                    "type" : "Feature",
                    "id" : row['County'],
                    "geometry" : ast.literal_eval(row['Geo Shape'])
                }
            )
    json.dump(geoDict, geoData)

In [9]:
def toPolyLine(geoShape):
    #Convert json polyline to folium polyline object
    jsonLine = json.loads(geoShape) #load json and get coordiantes
    polyLine = jsonLine['coordinates']

    #swap coordinate ordering for folium
    res = []
    for dim in polyLine:
        for pair in dim:
            res.append([pair[1], pair[0]])
    return res

df['polyline'] = df['Geo Shape'].dropna().apply(toPolyLine).dropna()

In [106]:
from colour import Color
import math

#normalize population
min, max = df['Total Population'].min(), df['Total Population'].max()
df['norm_population'] = df['Total Population'].apply(
    lambda x: (x-min)/(max-min)
)

#map population to color gradient
colorRange = list(Color("#FFEFEF").range_to(Color("#FF0000"), 100))

def toColor(population):
    size = len(colorRange) - 1
    index = math.floor(population*size)

    return colorRange[index].hex_l

df['gradient'] = df['norm_population'].map(toColor)

df['gradient'].iloc[100]

'#ffeded'

In [128]:
map = folium.Map(
    location=[40.8336, -74.5463], 
    zoom_start=10,
    )

def styleFunc(feature):
    print(feature['gradient'])            
    return {
        'fillColor': feature['gradient'],
        'fillOpacity': 1.0
    }

for index, row in df.iterrows():
    if row['Geo Shape'] is not np.nan:
        folium.GeoJson(
            data=str(row['Geo Shape']),
            style_function= styleFunc
            ).add_to(map)

KeyError: 'gradient'

In [39]:
with open('counties.json', 'r') as geoData:
    counties = json.load(geoData)

folium.Choropleth(
    geo_data=counties,
    name='choropleth',
    data=df,
    key_on='feature.id'
).add_to(map)

TypeError: 'NoneType' object is not subscriptable

In [133]:
map

In [119]:
df['gradient'].astype(str)

0       #ffefef
1       #ffefef
2       #ffefef
3       #ffdcdc
4       #ffefef
         ...   
3138    #ffefef
3139    #ffdcdc
3140    #ffefef
3141    #ffefef
3142    #ffefef
Name: gradient, Length: 3143, dtype: object